In [1]:
import sys
import pandas as pd
import json
import numpy as np
import torch
from torch import Tensor
import pickle
from tqdm.auto import tqdm

from matplotlib import pyplot as plt

sys.path.append("../")
from src.log import myLogger
from src.repository.data_repository import DataRepository
from src.checkpoint.checkpoint import Checkpoint
from src.metrics.jaccard import jaccard, calc_jaccard_mean

pd.set_option("display.max_columns", 300)
pd.set_option("display.max_rows", 600)

%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%config InlineBackend.figure_formats = {'png', 'retina'}

In [2]:
logger = myLogger("../logs/temp.log", exp_id="e000", wdb_prj_id="temp", exp_config=None, use_wdb=False) 
dr = DataRepository(logger=logger, local_root_path="..")

2021-11-15 17:20:29,069 log.py               31   [INFO] [__init__] skip wandb init 


## optuna ensemble

In [3]:
import pickle

with open("../data/dataset/optuna_ens/prep_dfs.pkl", "rb") as fin:
    prep_dfs = pickle.load(fin)

In [4]:
for k in prep_dfs:
    prep_dfs[k] = prep_dfs[k][prep_dfs[k].start_logits.notnull()]

In [5]:
optuna_exp_ids = ["e048", "e049", "e051", "e053", "e054", "e073", "e074"]

In [6]:
from src.postprocessor.factory import PostprocessorFactory

postprocessor = PostprocessorFactory(
    **{
        "postprocessor_type": "baseline_kernel",
        "n_best_size": 50,
        "max_answer_length": 30,
        "use_chars_length": False,
        "text_postprocess": "mypospro_ver1",
        "use_multiprocess": True,
    },
    logger=logger
).create()

In [7]:
def objective(trial):
    samples = [1./len(optuna_exp_ids)]*len(optuna_exp_ids)
    for i in range(len(optuna_exp_ids)):
        samples[i] = trial.suggest_uniform(optuna_exp_ids[i], 0, 1)
    
    w = [samples[i]/sum(samples) for i in range(len(samples))]
    
    ids = prep_dfs[optuna_exp_ids[0]]["id"]
    contexts = prep_dfs[optuna_exp_ids[0]]["context"]
    answer_texts = prep_dfs[optuna_exp_ids[0]]["answer_text"]
    offset_mappings = prep_dfs[optuna_exp_ids[0]]["offset_mapping"]
    
    start_logits = w[0] * prep_dfs[optuna_exp_ids[0]]["start_logits"]
    end_logits = w[0] * prep_dfs[optuna_exp_ids[0]]["end_logits"]
    for i in range(1, len(optuna_exp_ids)):
        start_logits += w[i] * prep_dfs[optuna_exp_ids[i]]["start_logits"]
        end_logits += w[i] * prep_dfs[optuna_exp_ids[i]]["end_logits"]

    pospro_ids, pospro_answer_texts, pospro_answer_preds = postprocessor(
        ids=ids,
        contexts=contexts,
        answer_texts=answer_texts,
        offset_mappings=offset_mappings,
        start_logits=start_logits,
        end_logits=end_logits,
    )
    val_jaccard = calc_jaccard_mean(
        text_trues=pospro_answer_texts, text_preds=pospro_answer_preds
    )
    return val_jaccard

In [10]:
import optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2021-11-15 17:36:00,096] A new study created in memory with name: no-name-ca7e2c83-dd99-45b0-8c96-3f2890b3850f
2021-11-15 17:36:01,030 postprocessor.py     58   [INFO] [__call__] start postprocessing 


  0%|          | 0/1114 [00:00<?, ?it/s]

[I 2021-11-15 17:36:08,799] Trial 0 finished with value: 0.7177427462324235 and parameters: {'e048': 0.8393150794282638, 'e049': 0.10851854699591046, 'e051': 0.9622121276728473, 'e053': 0.40489834405439806, 'e054': 0.33426326760165037, 'e073': 0.2674024049202862, 'e074': 0.24603048641822745}. Best is trial 0 with value: 0.7177427462324235.
2021-11-15 17:36:09,764 postprocessor.py     58   [INFO] [__call__] start postprocessing 


  0%|          | 0/1114 [00:00<?, ?it/s]

[I 2021-11-15 17:36:17,626] Trial 1 finished with value: 0.7155598503331899 and parameters: {'e048': 0.07507078242189491, 'e049': 0.9763854072261603, 'e051': 0.2689735152526628, 'e053': 0.21468672588920823, 'e054': 0.16737477013055768, 'e073': 0.558163319233035, 'e074': 0.016935450298797616}. Best is trial 0 with value: 0.7177427462324235.
2021-11-15 17:36:18,570 postprocessor.py     58   [INFO] [__call__] start postprocessing 


...

[I 2021-11-15 17:48:31,440] Trial 97 finished with value: 0.7291872761688744 and parameters: {'e048': 0.11257638387219801, 'e049': 0.12082065582184219, 'e051': 0.4372356987566384, 'e053': 0.5435511024894096, 'e054': 0.3067379128046348, 'e073': 0.14776191672678207, 'e074': 0.8929333043228167}. Best is trial 72 with value: 0.730458969765523.
2021-11-15 17:48:32,431 postprocessor.py     58   [INFO] [__call__] start postprocessing 


  0%|          | 0/1114 [00:00<?, ?it/s]

[I 2021-11-15 17:48:38,838] Trial 98 finished with value: 0.7257312618062172 and parameters: {'e048': 0.03178607623644386, 'e049': 0.20586454488299963, 'e051': 0.4797305358087542, 'e053': 0.4851763433327164, 'e054': 0.30658895234605343, 'e073': 0.14474419287045762, 'e074': 0.7968552845646355}. Best is trial 72 with value: 0.730458969765523.
2021-11-15 17:48:39,824 postprocessor.py     58   [INFO] [__call__] start postprocessing 


  0%|          | 0/1114 [00:00<?, ?it/s]

[I 2021-11-15 17:48:46,242] Trial 99 finished with value: 0.7320026833828452 and parameters: {'e048': 0.11564797851833304, 'e049': 0.12944013885571104, 'e051': 0.44727985926495023, 'e053': 0.5508725369566504, 'e054': 0.046024691261880746, 'e073': 0.1761450136227768, 'e074': 0.89243590478673}. Best is trial 99 with value: 0.7320026833828452.


In [11]:
d=study.best_params 
v=study.best_value
print(v, d)

0.7320026833828452 {'e048': 0.11564797851833304, 'e049': 0.12944013885571104, 'e051': 0.44727985926495023, 'e053': 0.5508725369566504, 'e054': 0.046024691261880746, 'e073': 0.1761450136227768, 'e074': 0.89243590478673}
